In [ ]:
#Pipeline imports
%matplotlib inline
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import matplotlib.pyplot as plt
import wirc_drp.wirc_object as wo
import wirc_drp.constants as constants
from wirc_drp.utils import calibration, spec_utils as su, image_utils as iu, source_utils as src
from wirc_drp.masks import *
import copy

from astropy.io import fits
from astropy.stats import sigma_clipped_stats

%load_ext autoreload
%autoreload 2

# Load Spectral Extraction Results

In [ ]:
target_name = 'SN2020oi'
obs_date = '20200119'
bkg_sub = 'AB'
filter_name = 'J'

spec_cube = np.load('spec_cube_%s_%s_%s_%s.npy'%(target_name, obs_date, filter_name, bkg_sub) )

In [ ]:
spec_cube.shape

# Align the spectra in the cube

In [ ]:
#align

align_spec_cube = np.copy(spec_cube)
align_vec = spec_cube[0,0,1,:]
# for i in range(spec_cube.shape[0]):
align_spec_cube = su.align_spectral_cube(np.nan_to_num(spec_cube[:,:,:,:]), ref_trace = align_vec)

In [ ]:
# Next two cells are for AB dither only

In [ ]:
# A_spec_cube = np.zeros((spec_cube.shape[0]//8+1,4,4,3,161))
# B_spec_cube = np.zeros((spec_cube.shape[0]//8,4,4,3,161))

In [ ]:
# # fig,ax = plt.subplots(2,4, figsize = (20, 10))
# argmaxes = []
# A_spec_cube_aligned = np.copy(A_spec_cube)
# B_spec_cube_aligned = np.copy(B_spec_cube)

# colors = ['r','c','b','y']
# trace_name = ['UL','LR','UR', 'LL']
# # align_vec = A_spec_cube[0,0, 0,1,:]



# from scipy.ndimage import median_filter, shift
# for i in range(A_spec_cube.shape[0]):
#     resA =  su.align_spectral_cube(np.nan_to_num(A_spec_cube[i,:,:,:,:]), ref_trace = align_vec)
#     resB =  su.align_spectral_cube(np.nan_to_num(B_spec_cube[i,:,:,:,:]), ref_trace = align_vec)
# #     print(resA.shape)
#     A_spec_cube_aligned[i] = resA
#     B_spec_cube_aligned[i] = resB 
#     fig,ax = plt.subplots(2,4, figsize = (16,5))

#     for j in range(A_spec_cube.shape[2]):
#         for ang in range(4):
#             #######This block is trying to align spectra to peak
# #             vec_A = A_spec_cube[i,ang, j,1,:]
# #             vec_B = B_spec_cube[i,ang, j,1,:]
# #             vec_A = median_filter(vec_A, 3)
# #             vec_B = median_filter(vec_B, 3)
# #             dA = np.argmax(vec_A) - 122
# #             dB = np.argmax(vec_B) - 122
# # # #             A_spec_cube_aligned[i,ang,j,1,:] = median_filter(shift(A_spec_cube[i,ang,j,1,:], -dA), 3)
# # # #             B_spec_cube_aligned[i,ang,j,1,:] = median_filter(shift(B_spec_cube[i,ang,j,1,:], -dB), 3)
# #             A_spec_cube_aligned[i,ang,j,1,:] = shift(A_spec_cube[i,ang,j,1,:], -dA)
# #             B_spec_cube_aligned[i,ang,j,1,:] = shift(B_spec_cube[i,ang,j,1,:], -dB)
#             ########################################
#             ax[0,j].plot(A_spec_cube_aligned[i,ang,j,1,:], color = colors[ang])
#             ax[0,j].axvline(105, ls = ':')
#             ax[0,j].axvline(80, ls = ':')
#             ax[0,j].axhline(0, ls = ':')

# #             if i != 1:
#             ax[1,j].plot(B_spec_cube_aligned[i,ang,j,1,:], color = colors[ang])
#             ax[1,j].axvline(105, ls = ':')
#             ax[1,j].axvline(80, ls = ':')
#             ax[1,j].axhline(0, ls = ':')

#             ax[0,j].set_ylim([-1000,6500])
#             ax[1,j].set_ylim([-1000,6500])
#             if i == 0:
#                 ax[0,j].set_title('%s trace'%trace_name[j])
#             ax[0,0].set_ylabel('A pos, set %d'%(i+1))
#             ax[1,0].set_ylabel('B pos, set %d'%(i+1))
#     plt.show()


# Compute wavelength solution

In [ ]:
flux_spec = np.median(align_spec_cube[:,0,1], axis = 0)

flux_spec = flux_spec/np.max(flux_spec)
# flux_spec[flux_spec > 0.2] = 0.2

wlSol_rough = su.rough_wavelength_calibration_v2(flux_spec, filter_name = filter_name)



In [ ]:
from wirc_drp import constants

#This line fiddle with the wavelength solution to correct it
# wlSol = ((wlSol_rough-1.145+0.005)*0.98)+1.145
wlSol = wlSol_rough


plt.plot(wlSol, flux_spec/np.max(flux_spec))
lb,dlb, f0, transmission, central_wl_pix = constants.getFilterInfo('J')
plt.plot(wlSol, transmission(wlSol))

# plt.axvline(1.31)

In [ ]:
align_spec_cube.shape

# Load a half wave plate sequence and compute q and u

In [ ]:
HWP_all = np.load('HWP_%s_%s_%s.npy'%(target_name, obs_date, filter_name))

In [ ]:
# A_q, A_u, A_qerr, A_uerr, A_qind, A_uind = src.compute_qu_for_obs_sequence(A_spec_cube_aligned, HWP_A)
# B_q, B_u, B_qerr, B_uerr, B_qind, B_uind = src.compute_qu_for_obs_sequence(B_spec_cube_Bligned, HWP_B)

#Compute q and u here

all_q, all_u, all_qerr, all_uerr, all_qind, all_uind = \
    src.compute_qu_for_obs_sequence(align_spec_cube , np.array(HWP_all), run_alignment = False)

In [ ]:
# q_A = []
# q_B = []
# u_A = []
# u_B = []
# for i in range(len(all_q)):
#     if i%8 <= 3:
#         q_A += [all_q[i]]
#         u_A += [all_u[i]]
#     else:
#         q_B += [all_q[i]]
#         u_B += [all_u[i]] 

# Organize results into those from LL-UR and LR-UL pairs for calibration
compare results from LL-UR vs LR-UL pairs

In [ ]:
q_ind0 = all_q[all_qind == 0]
q_ind1 = all_q[all_qind == 1]
u_ind0 = all_u[all_uind == 0]
u_ind1 = all_u[all_uind == 1]

In [ ]:
#calibrate_qu(wvs,q,u,qerr,uerr,trace_pair)

In [ ]:
P0q_sc,P0q_sc_med,P0q_sc_std = sigma_clipped_stats(q_ind0, sigma = 3, maxiters = 5, axis = 0)
P0u_sc,P0u_sc_med,P0u_sc_std = sigma_clipped_stats(u_ind0, sigma = 3, maxiters = 5, axis = 0)
P1q_sc,P1q_sc_med,P1q_sc_std = sigma_clipped_stats(q_ind1, sigma = 3, maxiters = 5, axis = 0)
P1u_sc,P1u_sc_med,P1u_sc_std = sigma_clipped_stats(u_ind1, sigma = 3, maxiters = 5, axis = 0)

## Calibrate for polarimetric efficiency and cross-talk

In [ ]:
#Calibrate q u efficiency and cross-talk
cal_q0,cal_u0,cal_q_err0,cal_u_err0 = calibration.calibrate_qu(wlSol, P0q_sc, P0u_sc,
                                         P0q_sc_std/np.sqrt(len(q_ind0)), P0u_sc_std/np.sqrt(len(u_ind0)), 0)

cal_q1,cal_u1,cal_q_err1,cal_u_err1 = calibration.calibrate_qu(wlSol, P1q_sc, P1u_sc,
                                         P1q_sc_std/np.sqrt(len(q_ind1)), P1u_sc_std/np.sqrt(len(u_ind1)), 1)

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (18,9))

# for i in range(len(q_ind0)):
#     ax[0].plot(q_ind0[i]*100, 'b', alpha = 0.1)
#     ax[1].plot(u_ind0[i]*100, 'b', alpha = 0.1)
# for i in range(len(q_ind1)):
#     ax[0].plot(q_ind1[i]*100, 'r', alpha = 0.1)
#     ax[1].plot(u_ind1[i]*100, 'r', alpha = 0.1)
    
ax[0].errorbar(range(len(P0q_sc)), P0q_sc*100, yerr = P0q_sc_std*100/np.sqrt(len(q_ind0)), color ='b', alpha = 0.5)
ax[1].errorbar(range(len(P0u_sc)), -P0u_sc*100, yerr = P0u_sc_std*100/np.sqrt(len(u_ind0)), color ='b', alpha = 0.5)
ax[0].errorbar(range(len(P1q_sc)), P1q_sc*100, yerr = P1q_sc_std*100/np.sqrt(len(q_ind1)), color ='r', alpha = 0.5)
ax[1].errorbar(range(len(P1u_sc)), -P1u_sc*100, yerr = P1u_sc_std*100/np.sqrt(len(u_ind1)), color ='r', alpha = 0.5)

ax[0].errorbar(range(len(P0q_sc)), cal_q0*100, yerr = cal_q_err0, color ='b', alpha = 1)
ax[1].errorbar(range(len(P0u_sc)), cal_u0*100, yerr = cal_u_err0, color ='b', alpha = 1)
ax[0].errorbar(range(len(P1q_sc)), cal_q1*100, yerr = cal_q_err1, color ='r', alpha = 1)
ax[1].errorbar(range(len(P1u_sc)), cal_u1*100, yerr = cal_u_err1, color ='r', alpha = 1)
    
ax[0].set_ylim([-6,6])
ax[1].set_ylim([-6,6])
ax[0].set_xlim([40,120])
ax[1].set_xlim([40,120])


## Non-smooth AB diagnostic
Use this only for AB subtraction to compare if results from A and B agree

In [ ]:
# Aq_sc,Aq_sc_med,Aq_sc_std = sigma_clipped_stats(q_A, sigma = 3, maxiters = 5, axis = 0)
# Au_sc,Au_sc_med,Au_sc_std = sigma_clipped_stats(u_A, sigma = 3, maxiters = 5, axis = 0)

# Bq_sc,Bq_sc_med,Bq_sc_std = sigma_clipped_stats(q_B, sigma = 3, maxiters = 5, axis = 0)
# Bu_sc,Bu_sc_med,Bu_sc_std = sigma_clipped_stats(u_B, sigma = 3, maxiters = 5, axis = 0)

In [ ]:
# fig, ax = plt.subplots(1,2,figsize = (18,9))

# for i in range(len(q_A)):
#     ax[0].plot(q_A[i]*100, 'b', alpha = 0.1)
#     ax[1].plot(u_A[i]*100, 'b', alpha = 0.1)
# for i in range(len(q_B)):
#     ax[0].plot(q_B[i]*100, 'r', alpha = 0.1)
#     ax[1].plot(u_B[i]*100, 'r', alpha = 0.1)
    
# ax[0].errorbar(range(len(Aq_sc)), Aq_sc*100, yerr = Aq_sc_std*100/np.sqrt(len(q_A)), color ='b', alpha = 1)
# ax[1].errorbar(range(len(Au_sc)), Au_sc*100, yerr = Au_sc_std*100/np.sqrt(len(u_A)), color ='b', alpha = 1)
# ax[0].errorbar(range(len(Bq_sc)), Bq_sc*100, yerr = Bq_sc_std*100/np.sqrt(len(q_B)), color ='r', alpha = 1)
# ax[1].errorbar(range(len(Bu_sc)), Bu_sc*100, yerr = Bu_sc_std*100/np.sqrt(len(u_B)), color ='r', alpha = 1)
    
# ax[0].set_ylim([-10,10])
# ax[1].set_ylim([-10,10])
# ax[0].set_xlim([40,120])
# ax[1].set_xlim([40,120])


In [ ]:
# from astropy.stats import sigma_clipped_stats

# q_sc,q_sc_med,q_sc_std = sigma_clipped_stats(all_q, sigma = 3, maxiters = 5, axis = 0)
# u_sc,u_sc_med,u_sc_std = sigma_clipped_stats(all_u, sigma = 3, maxiters = 5, axis = 0)


In [ ]:
# fig, ax = plt.subplots(1,2,figsize = (18,9))

# ax[0].errorbar(range(len(q_sc)), q_sc*100, q_sc_std*100/np.sqrt(len(all_q)))
# ax[0].set_ylim([-10,10])
# ax[0].set_xlim([40,125])


# ax[1].errorbar(range(len(u_sc)), u_sc*100, u_sc_std*100/np.sqrt(len(all_u)))
# ax[1].set_ylim([-10,10])
# ax[1].set_xlim([40,120])


# #A and B
# ax[0].errorbar(range(len(Aq_sc)), Aq_sc*100, yerr = Aq_sc_std*100/np.sqrt(len(q_A)), color ='b', alpha = 0.5)
# ax[1].errorbar(range(len(Au_sc)), Au_sc*100, yerr = Au_sc_std*100/np.sqrt(len(u_A)), color ='b', alpha = 0.5)
# ax[0].errorbar(range(len(Bq_sc)), Bq_sc*100, yerr = Bq_sc_std*100/np.sqrt(len(q_B)), color ='r', alpha = 0.5)
# ax[1].errorbar(range(len(Bu_sc)), Bu_sc*100, yerr = Bu_sc_std*100/np.sqrt(len(u_B)), color ='r', alpha = 0.5)


# ax[0].axhline(0)
# ax[1].axhline(0)

In [ ]:
np.stack([cal_q0, cal_q1]).shape

## Smooth spectra

In [ ]:
#bin size
bs = 5

# cal_q = np.average(np.stack([cal_q0, cal_q1]), weights = np.stack([cal_q_err0, cal_q_err1]), axis = 0)
# cal_u = np.average(np.stack([cal_u0, cal_u1]), weights = np.stack([cal_u_err0, cal_u_err1]), axis = 0)

# cal_q_err = np.sqrt(np.mean(np.stack([cal_q_err0**2, cal_q_err1**2]), axis = 0))
# cal_u_err = np.sqrt(np.mean(np.stack([cal_u_err0**2, cal_u_err1**2]), axis = 0))


smooth_q = su.smooth_spectra(all_q, kernel = 'box', smooth_size=bs, rebin = True)
smooth_u = su.smooth_spectra(all_u, kernel = 'box', smooth_size=bs, rebin = True)



q_sc_sm,q_sc_med_sm,q_sc_std_sm = sigma_clipped_stats(smooth_q, sigma = 3, maxiters = 5, axis = 0)
u_sc_sm,u_sc_med_sm,u_sc_std_sm = sigma_clipped_stats(smooth_u, sigma = 3, maxiters = 5, axis = 0)

wlSol_smooth = su.smooth_spectra(wlSol, kernel = 'box', smooth_size = bs, rebin = True)


## Smooth pairs 1&2 diagnostic
pair 1 is observed at HWP angles 0 and 45 deg; pair 2 at 22.5 and 67.5 deg

In [ ]:
q_sm_ind0 = smooth_q[all_qind == 0]
q_sm_ind1 = smooth_q[all_qind == 1]
u_sm_ind0 = smooth_u[all_uind == 0]
u_sm_ind1 = smooth_u[all_uind == 1]

In [ ]:
# #Calibrate q u efficiency and cross-talk
# cal_q0_sm,cal_u0_sm,cal_q_err0_sm,cal_u_err0_sm = calibration.calibrate_qu(wlSol_smooth, P0q_sc_sm, P0u_sc_sm,
#                                          P0q_sc_std_sm/np.sqrt(len(q_sm_ind0)), P0u_sc_std_sm/np.sqrt(len(u_sm_ind0)), 0)

# cal_q1_sm,cal_u1_sm,cal_q_err1_sm,cal_u_err1_sm = calibration.calibrate_qu(wlSol_smooth, P1q_sc_sm, P1u_sc_sm,
#                                          P1q_sc_std_sm/np.sqrt(len(q_sm_ind1)), P1u_sc_std_sm/np.sqrt(len(u_sm_ind1)), 1)

In [ ]:
P0q_sc_sm,P0q_sc_med_sm,P0q_sc_std_sm = sigma_clipped_stats(q_sm_ind0, sigma = 3, maxiters = 20, axis = 0)
P0u_sc_sm,P0u_sc_med_sm,P0u_sc_std_sm = sigma_clipped_stats(u_sm_ind0, sigma = 3, maxiters = 20, axis = 0)
P1q_sc_sm,P1q_sc_med_sm,P1q_sc_std_sm = sigma_clipped_stats(q_sm_ind1, sigma = 3, maxiters = 20, axis = 0)
P1u_sc_sm,P1u_sc_med_sm,P1u_sc_std_sm = sigma_clipped_stats(u_sm_ind1, sigma = 3, maxiters = 20, axis = 0)

## Calibrate efficiency and cross talk

In [ ]:
#Calibrate q u efficiency and cross-talk
cal_q0_sm,cal_u0_sm,cal_q_err0_sm,cal_u_err0_sm = calibration.calibrate_qu(wlSol_smooth, P0q_sc_sm, P0u_sc_sm,
                                         P0q_sc_std_sm/np.sqrt(len(q_sm_ind0)), P0u_sc_std_sm/np.sqrt(len(u_sm_ind0)), 0)

cal_q1_sm,cal_u1_sm,cal_q_err1_sm,cal_u_err1_sm = calibration.calibrate_qu(wlSol_smooth, P1q_sc_sm, P1u_sc_sm,
                                         P1q_sc_std_sm/np.sqrt(len(q_sm_ind1)), P1u_sc_std_sm/np.sqrt(len(u_sm_ind1)), 1)

In [ ]:
# P0q_sc_sm

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (18,9))

for i in range(len(q_sm_ind0)):
    ax[0].plot(q_sm_ind0[i]*100, 'b', alpha = 0.1)
    ax[1].plot(u_sm_ind0[i]*100, 'b', alpha = 0.1)
for i in range(len(q_sm_ind1)):
    ax[0].plot(q_sm_ind1[i]*100, 'r', alpha = 0.1)
    ax[1].plot(u_sm_ind1[i]*100, 'r', alpha = 0.1)
    
ax[0].errorbar(range(len(P0q_sc_sm)), P0q_sc_sm*100, yerr = P0q_sc_std_sm*100/np.sqrt(len(q_sm_ind0)), color ='b', alpha = 0.5)
ax[1].errorbar(range(len(P0u_sc_sm)), P0u_sc_sm*100, yerr = P0u_sc_std_sm*100/np.sqrt(len(u_sm_ind0)), color ='b', alpha = 0.5)
ax[0].errorbar(range(len(P1q_sc_sm)), P1q_sc_sm*100, yerr = P1q_sc_std_sm*100/np.sqrt(len(q_sm_ind1)), color ='r', alpha = 0.5)
ax[1].errorbar(range(len(P1u_sc_sm)), P1u_sc_sm*100, yerr = P1u_sc_std_sm*100/np.sqrt(len(u_sm_ind1)), color ='r', alpha = 0.5)

ax[0].errorbar(range(len(P0q_sc_sm)), cal_q0_sm*100, yerr = cal_q_err0_sm, color ='b', alpha = 1)
ax[1].errorbar(range(len(P0u_sc_sm)), cal_u0_sm*100, yerr = cal_u_err0_sm, color ='b', alpha = 1)
ax[0].errorbar(range(len(P1q_sc_sm)), cal_q1_sm*100, yerr = cal_q_err1_sm, color ='r', alpha = 1)
ax[1].errorbar(range(len(P1u_sc_sm)), cal_u1_sm*100, yerr = cal_u_err1_sm, color ='r', alpha = 1)

ax[0].set_ylim([-10,10])
ax[1].set_ylim([-10,10])
ax[0].set_xlim([40//5,120//5])
ax[1].set_xlim([40//5,120//5])


## Smooth AB positions comparison

In [ ]:
# q_A_sm = []
# q_B_sm = []
# u_A_sm = []
# u_B_sm = []
# for i in range(len(all_q)):
#     if i%8 <= 3:
#         q_A_sm += [smooth_q[i]]
#         u_A_sm += [smooth_u[i]]
#     else:
#         q_B_sm += [smooth_q[i]]
#         u_B_sm += [smooth_u[i]] 

In [ ]:
# qA_sc_sm,qA_sc_med_sm,qA_sc_std_sm = sigma_clipped_stats(q_A_sm, sigma = 3, maxiters = 5, axis = 0)
# uA_sc_sm,uA_sc_med_sm,uA_sc_std_sm = sigma_clipped_stats(u_A_sm, sigma = 3, maxiters = 5, axis = 0)
# qB_sc_sm,qB_sc_med_sm,qB_sc_std_sm = sigma_clipped_stats(q_B_sm, sigma = 3, maxiters = 5, axis = 0)
# uB_sc_sm,uB_sc_med_sm,uB_sc_std_sm = sigma_clipped_stats(u_B_sm, sigma = 3, maxiters = 5, axis = 0)

In [ ]:
# fig, ax = plt.subplots(1,2,figsize = (18,9))

# for i in range(len(q_A_sm)):
#     ax[0].plot(q_A_sm[i]*100, 'b', alpha = 0.1)
#     ax[1].plot(u_A_sm[i]*100, 'b', alpha = 0.1)
# for i in range(len(q_B_sm)):
#     ax[0].plot(q_B_sm[i]*100, 'r', alpha = 0.1)
#     ax[1].plot(u_B_sm[i]*100, 'r', alpha = 0.1)
    
# ax[0].errorbar(range(len(qA_sc_sm)), qA_sc_sm*100, yerr = qA_sc_std_sm*100/np.sqrt(len(q_A_sm)), color ='b', alpha = 1)
# ax[1].errorbar(range(len(uA_sc_sm)), uA_sc_sm*100, yerr = uA_sc_std_sm*100/np.sqrt(len(u_A_sm)), color ='b', alpha = 1)
# ax[0].errorbar(range(len(qB_sc_sm)), qB_sc_sm*100, yerr = qB_sc_std_sm*100/np.sqrt(len(q_B_sm)), color ='r', alpha = 1)
# ax[1].errorbar(range(len(uB_sc_sm)), uB_sc_sm*100, yerr = uB_sc_std_sm*100/np.sqrt(len(u_B_sm)), color ='r', alpha = 1)

# # ax[0].errorbar(range(len(qA_sc_sm)), qA_sc_med_sm*100, yerr = qA_sc_std_sm*100/np.sqrt(len(q_A_sm)), color ='m', alpha = 1)
# # ax[1].errorbar(range(len(uA_sc_sm)), uA_sc_med_sm*100, yerr = uA_sc_std_sm*100/np.sqrt(len(u_A_sm)), color ='m', alpha = 1)
# # ax[0].errorbar(range(len(qB_sc_sm)), qB_sc_med_sm*100, yerr = qB_sc_std_sm*100/np.sqrt(len(q_B_sm)), color ='g', alpha = 1)
# # ax[1].errorbar(range(len(uB_sc_sm)), uB_sc_med_sm*100, yerr = uB_sc_std_sm*100/np.sqrt(len(u_B_sm)), color ='g', alpha = 1)


# ax[0].set_ylim([-10,10])
# ax[1].set_ylim([-10,10])
# ax[0].set_xlim([40//5,120//5])
# ax[1].set_xlim([40//5,120//5])


## Final q and u:  average calibrated q and u

In [ ]:
# cal_q = np.average(np.stack([cal_q0_sm, cal_q1_sm]), weights = np.stack([cal_q_err0_sm, cal_q_err1_sm]), axis = 0)
# cal_u = np.average(np.stack([cal_u0_sm, cal_u1_sm]), weights = np.stack([cal_u_err0_sm, cal_u_err1_sm]), axis = 0)

cal_q_sm = np.mean([cal_q0_sm, cal_q1_sm], axis = 0)
cal_u_sm = np.mean([cal_u0_sm, cal_u1_sm], axis = 0)

cal_q_err_sm = np.sqrt(cal_q_err0_sm**2 + cal_q_err1_sm**2)/2
cal_u_err_sm = np.sqrt(cal_u_err0_sm**2 + cal_u_err1_sm**2)/2

## Plot results

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (15,6))

wlMin = 1.17
wlMax = 1.32
plot_range = np.where( np.logical_and(wlSol_smooth < wlMax , wlSol_smooth > wlMin))

#smooth
# bs = 3
# sm_q = su.smooth_spectra(q_sc, kernel = 'box', smooth_size = bs, rebin = True)
# sm_u = su.smooth_spectra(u_sc, kernel = 'box', smooth_size = bs, rebin = True)

# sm_q_std = su.smooth_spectra(q_sc_std, kernel = 'box', smooth_size = bs, rebin = True)
# sm_u_std = su.smooth_spectra(u_sc_std, kernel = 'box', smooth_size = bs, rebin = True)




# ax[0].errorbar(wlSol_smooth, sm_q*100, sm_q_std*100/np.sqrt(len(all_q)), marker = 'o', ls = 'None')
ax[0].set_ylim([-3,7])
ax[0].set_xlim([wlMin,wlMax])

# ax[0].errorbar(wlSol_smooth, q_sc_sm*100, q_sc_std_sm*100/np.sqrt(len(all_q)), marker = 's', ls = 'None')
ax[0].errorbar(wlSol_smooth, cal_q_sm*100, cal_q_err_sm*100, marker = 's', ls = 'None')



# ax[1].errorbar(wlSol_smooth, sm_u*100, sm_u_std*100/np.sqrt(len(all_u)), marker = 'o', ls = 'None')
ax[1].set_ylim([-3,7])
ax[1].set_xlim([wlMin,wlMax])

ax[1].errorbar(wlSol_smooth, cal_u_sm*100, cal_u_err_sm*100, marker = 's', ls = 'None')


#Flux
ax[0].plot(wlSol, flux_spec/np.max(flux_spec)*6)
ax[1].plot(wlSol, flux_spec/np.max(flux_spec)*6)


ax[0].axhline(0, ls = '--', color = 'k')
ax[1].axhline(0, ls = '--', color = 'k')

ax[0].tick_params(labelsize = 18)
ax[1].tick_params(labelsize = 18)


ax[0].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[0].set_ylabel('q (%)', fontsize = 20)
ax[0].text(1.22,6,'Normalized Flux', color = 'C1', fontsize = 18)


ax[1].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[1].set_ylabel('u (%)', fontsize = 20)
ax[1].text(1.22,6,'Normalized Flux', color = 'C1', fontsize = 18)


# plt.savefig('20oi_qu_prelim_align_peak.pdf')

# Plot q and u in the unit of standard deviation to judge detection

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (15,6))

wlMin = 1.17
wlMax = 1.31


# ax[0].errorbar(wlSol_smooth, sm_q*100, sm_q_std*100/np.sqrt(len(all_q)), marker = 'o', ls = 'None')
ax[0].set_ylim([-5,5])
ax[0].set_xlim([wlMin,wlMax])

ax[0].plot(wlSol_smooth, q_sc_sm/(q_sc_std_sm/np.sqrt(len(all_q))), marker = 's', ls = 'None')



# ax[1].errorbar(wlSol_smooth, sm_u*100, sm_u_std*100/np.sqrt(len(all_u)), marker = 'o', ls = 'None')
ax[1].set_ylim([-5,5])
ax[1].set_xlim([wlMin,wlMax])

ax[1].plot(wlSol_smooth, u_sc_sm/(u_sc_std_sm/np.sqrt(len(all_u))), marker = 's', ls = 'None')


#Flux
# ax[0].plot(wlSol, flux_spec/np.max(flux_spec)*6)
# ax[1].plot(wlSol, flux_spec/np.max(flux_spec)*6)


ax[0].axhline(0, ls = '--', color = 'k')
ax[1].axhline(0, ls = '--', color = 'k')

ax[0].tick_params(labelsize = 18)
ax[1].tick_params(labelsize = 18)


ax[0].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[0].set_ylabel('Measured q ($\sigma$)', fontsize = 20)
# ax[0].text(1.22,6,'Normalized Flux', color = 'C1', fontsize = 18)


ax[1].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[1].set_ylabel('Measured u ($\sigma$)', fontsize = 20)
# ax[1].text(1.22,6,'Normalized Flux', color = 'C1', fontsize = 18)
good_ind = np.logical_or(q_sc_sm/(q_sc_std_sm/np.sqrt(len(all_q))) > 3, u_sc_sm/(u_sc_std_sm/np.sqrt(len(all_u)))>2.9)

ax[0].axhline(3)
ax[1].axhline(2.9)

# plt.savefig('%s_qu_sigma.pdf'%SN_name)
# np.save('%s_good_ind.npy'%SN_name, good_ind)

In [ ]:
# good_ind

# qu Plane Plot

In [ ]:
# from matplotlib import cm

# # nearby_pol = np.load('../2017eaw/new_nearby_pol.npy')
# # nearby_wlSol = np.load('../2017eaw/new_nearby_wlSol.npy')

# # #BD+59 from June
# # BD_pol = np.load('../20180623_2017eaw_cal/june_cal_BD59_qu.npy')


# plt.figure(figsize = (7,6))
# SN_range = np.where( np.logical_and(wlSol_smooth < wlMax , wlSol_smooth > wlMin))


# cm1 = plt.get_cmap('plasma')


# col_SN = cm1( (wlSol_smooth[SN_range] - 1.2) /len(SN_range))




# #SN points
# SN_color = wlSol_smooth[SN_range]#(nearby_wlSol[nearby_range] - 1.18)/len(nearby_range)
# SN_qu = plt.scatter(100*cal_q_sm[SN_range], 100*cal_u_sm[SN_range], c = SN_color, 
#                             marker= 'o', cmap = cm.plasma,s  =100)
# print(SN_qu.get_facecolor())

# cb = plt.colorbar(SN_qu)

# cb.ax.tick_params(labelsize=18)
# cb.set_label('Wavelength ($\mu$m)', fontsize = 18)

# #SN line and errorbars
# lines, foo,errorlinecollection = plt.errorbar(100*cal_q_sm[SN_range], 100*cal_u_sm[SN_range], \
#               xerr = 100*cal_q_err_sm[SN_range], yerr = 100*cal_u_err_sm[SN_range], 
#               fmt= '--')#, ecolor = SN_qu.get_facecolor())
# # error_color = cb.to_rgba(SN_color)
# # error_color = matplotlib.colors.to_rgba_array(SN_color)
# ecols = cm.plasma( (wlSol_smooth[SN_range]-np.min(wlSol_smooth[SN_range]))/ \
#                   np.max((wlSol_smooth[SN_range]-np.min(wlSol_smooth[SN_range])))  )

# errorlinecollection[0].set_color(ecols)
# errorlinecollection[1].set_color(ecols)


# # print(errorlinecollection[0].get_color())
# # print(errorlinecollection[1].get_color())
                    


# plt.tick_params(axis='both', which='major', labelsize=18)
# plt.xlabel('q (%)', fontsize = 18)
# plt.ylabel('u (%)', fontsize = 18)

# # plt.text(0.7,1.1,'SN', fontsize = 18)
# # plt.text(-2.5,0.3,'J2034 \n2017 June ', fontsize = 18)
# # plt.text(-2.7,-2.5,'BD+59 2267 \n2018 June ', fontsize = 18)


# #plt.errorbar(np.median(q_ISP)*100, np.median(u_ISP)*100)#, xerr = np.std(q_ISP)*100, yerr = np.std(u_ISP)*100)
# #plt.errorbar(np.median(q_ISP_2)*100, np.median(u_ISP_2)*100)#, xerr = np.std(q_ISP_2)*100, yerr =np.std(u_ISP_2)*100)

# # plt.plot(np.median(q_ISP)*100, np.median(u_ISP)*100, marker = 'x', markersize = 20, mew = 5, color = 'C0')
# # plt.plot(np.median(q_ISP_2)*100, np.median(u_ISP_2)*100, marker = 'x', markersize = 20, mew = 5, color = 'C1')

# # plt.text(-1.4, -0.8, "ISP\nmodel A", fontsize = 18, color = 'C0', ha = 'right')
# # plt.text(0.2, -1.9, "ISP model B", fontsize = 18, color = 'C1')

# #hightlight 0,0
# plt.plot([0,0], [-3,3], color = 'k', alpha = 0.3)
# plt.plot([-3,3],[0,0] , color = 'k', alpha = 0.3)


# plt.grid(ls = '--')
# plt.xlim([-3,3])
# plt.ylim([-3,3])


# # plt.savefig('2018hna_raw_qu_aligned_peak.pdf')

In [ ]:
# #NIRES Spectrum
# from astropy.io import ascii as asci
# # NIRES_spec = asci.read('nir_spectra/18hna_mar_NIRES.ascii', names = ['wl', 'flux', 'eflux'])
# NIRES_spec = asci.read('nir_spectra/18hna_may_NIRES.ascii', names = ['wl', 'flux'])

# NIRES_range = np.logical_and(NIRES_spec['wl'] > wlMin*10000, NIRES_spec['wl'] < 1.32*10000)

In [ ]:
# plt.plot(NIRES_spec[NIRES_range]['wl'], NIRES_spec[NIRES_range]['flux'])

In [ ]:
# np.save('%s_normalized_flux.npy'%SN_name, np.array([ wlSol, flux_spec/np.max(flux_spec)]))

# p and $\theta$

In [ ]:
p_old, p_corr_old, dp_old, theta_old, dtheta_old = src.compute_p_and_pa(q_sc_sm, u_sc_sm,q_sc_std_sm/np.sqrt(len(all_q)), u_sc_std_sm/np.sqrt(len(all_u)))

p, p_corr, dp, theta, dtheta = src.compute_p_and_pa(cal_q_sm, cal_u_sm ,cal_q_err_sm, cal_u_err_sm)


theta[theta<0] = theta[theta<0]+np.pi
# theta[theta>np.pi] = theta[theta>np.pi]-np.pi

In [ ]:
plt.errorbar(wlSol_smooth, 100*p_old, 100*dp_old, color = 'r', ls = 'None')
plt.errorbar(wlSol_smooth, 100*p,     100*dp, color = 'k', ls = 'None')
plt.xlim([wlMin, wlMax])
plt.ylim([0,5])

In [ ]:
plt.plot(wlSol_smooth, 100*dp_old, color = 'r', ls = 'None', marker = 'o')
plt.plot(wlSol_smooth, 100*dp, color = 'k', ls = 'None', marker = 'o')
plt.xlim([wlMin, wlMax])
plt.ylim([0,1])

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (15,6))


ax[0].errorbar(wlSol_smooth, p*100, dp*100, marker = 'o', ls = 'None', alpha = 0.5)
ax[0].errorbar(wlSol_smooth, p_corr*100, dp*100, marker = 'o', ls = 'None')

ax[0].set_ylim([-1,5])
ax[0].set_xlim([wlMin,wlMax])

thetadeg = np.degrees(theta)
# thetadeg[thetadeg > 150] = thetadeg[thetadeg > 150]-180
ax[1].errorbar(wlSol_smooth, thetadeg, np.degrees(dtheta), marker = 'o', ls = 'None')
# ax[1].set_ylim([50,200])
ax[1].set_xlim([wlMin,wlMax])

#Flux
ax[0].plot(wlSol, flux_spec/np.max(flux_spec)*4)
ax[1].plot(wlSol, flux_spec/np.max(flux_spec)*180)


ax[0].axhline(0, ls = '--', color = 'k')
ax[1].axhline(0, ls = '--', color = 'k')

ax[0].tick_params(labelsize = 18)
ax[1].tick_params(labelsize = 18)


ax[0].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[0].set_ylabel('p (%)', fontsize = 20)


ax[1].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
ax[1].set_ylabel('theta (deg)', fontsize = 20)


# plt.savefig('%s_ptheta_prelim_align_peak.pdf'%SN_name)

In [ ]:
#Save P
# np.save('%s_ptheta.npy'%SN_name,np.array([wlSol_smooth, p_corr*100, dp*100, np.degrees(theta), np.degrees(dtheta)]))

## Summary Figure

In [ ]:
# def annotateLine(ax, wl, y, length, line_name, textoffset, color, show_text = True, **kwargs):
#     ###draw line
#     ax.plot([wl,wl], [y, y-length], color = color)
#     if show_text:
#         ax.text(wl-textoffset[0], y-length-textoffset[1], line_name, color = color, **kwargs, va = 'top', ha ='center')
# #     ax.annotate(line_name,xy = (wl, y-length), xytext = (wl-textoffset[0], y-length-textoffset[1]), \
# #                      arrowprops = {'arrowstyle':'-'}, ha = 'center', va = 'top', **kwargs)


In [ ]:
# from matplotlib import cm
# fig, ax = plt.subplots(1,3,figsize = (24,6))

# wlMin = 1.17
# wlMax = 1.31


# # ax[0].errorbar(wlSol_smooth, sm_q*100, sm_q_std*100/np.sqrt(len(all_q)), marker = 'o', ls = 'None')
# ax[0].set_ylim([-2,4])
# ax[0].set_xlim([wlMin,wlMax])

# ax[0].errorbar(wlSol_smooth, cal_q_sm*100, cal_q_err_sm*100, \
#                marker = 's', ls = 'None', color = '#FF0018')

# ax[0].axhline(0, ls = '--', color = 'k')
# # ax[1].axhline(0, ls = '--', color = 'k')



# # ax[1].tick_params(labelsize = 18)

# ax[0].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
# ax[0].set_ylabel('q (%)', fontsize = 20, color = '#FF0018')

# #Plot u on the same axis
# u_ax = ax[0].twinx()
# # ax[1].errorbar(wlSol_smooth, sm_u*100, sm_u_std*100/np.sqrt(len(all_u)), marker = 'o', ls = 'None')
# u_ax.set_ylim([-2,4])
# u_ax.set_xlim([wlMin,wlMax])

# u_ax.errorbar(wlSol_smooth, cal_u_sm*100, cal_u_err_sm*100, \
#               marker = 's', ls = 'None', color = '#0000F9')

# #Ticks and labels

# ax[0].tick_params(labelsize = 18)
# ax[0].tick_params(axis = 'y', labelcolor = '#FF0018')

# u_ax.tick_params(labelsize = 18, labelcolor = '#0000F9')
# u_ax.set_ylabel('u (%)', fontsize = 20, color = '#0000F9')

# #Flux
# ax[0].plot(wlSol, (flux_spec/np.max(flux_spec) - 0.5)*6, color = '#FFA52C')
# # ax[1].plot(wlSol, flux_spec/np.max(flux_spec)*3.5, color = 'C1')

# #NIRES flux
# # ax[0].plot(NIRES_spec['wl'][NIRES_range]/10000, NIRES_spec['flux'][NIRES_range]/np.max(NIRES_spec['flux'][NIRES_range])*3.7, color = 'red')
# # ax[1].plot(NIRES_spec['wl'][NIRES_range]/10000, NIRES_spec['flux'][NIRES_range]/np.max(NIRES_spec['flux'][NIRES_range])*3.7, color = 'red')


# ax[0].text(1.19,3.5,'WIRC+Pol Normalized Flux', color = '#FFA52C', fontsize = 18)
# # ax[0].text(1.19,3.0,'NIRES Normalized Flux (May)', color = 'r', fontsize = 18)


# # ax[1].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
# # ax[1].set_ylabel('u (%)', fontsize = 20)
# # ax[1].text(1.19,3.5,'WIRC+Pol Normalized Flux', color = 'C1', fontsize = 18)
# # ax[1].text(1.19,3.0,'NIRES Normalized Flux', color = 'r', fontsize = 18)

# #qu plane
# SN_range = np.where( np.logical_and(wlSol_smooth < wlMax , wlSol_smooth > wlMin))


# cm1 = plt.get_cmap('plasma')


# col_SN = cm1( (wlSol_smooth[SN_range] - 1.2) /len(SN_range))


# #SN points
# SN_color = wlSol_smooth[SN_range]#(nearby_wlSol[nearby_range] - 1.18)/len(nearby_range)
# SN_qu = ax[1].scatter(100*cal_q_sm[SN_range], 100*cal_u_sm[SN_range], c = SN_color, \
#                             marker= 'o', cmap = cm.plasma,s  =100)


# #SN line and errorbars
# lines, foo,errorlinecollection = ax[1].errorbar(100*cal_q_sm[SN_range], 100*cal_u_sm[SN_range], \
#               xerr = 100*cal_q_err_sm[SN_range], yerr = 100*cal_u_err_sm[SN_range], fmt= '--')
# ecols = cm.plasma( (wlSol_smooth[SN_range]-np.min(wlSol_smooth[SN_range]))/ \
#                   np.max((wlSol_smooth[SN_range]-np.min(wlSol_smooth[SN_range])))  )

# errorlinecollection[0].set_color(ecols)
# errorlinecollection[1].set_color(ecols)

# cb = fig.colorbar(SN_qu, ax = ax[1])
# cb.ax.tick_params(labelsize=18)
# cb.set_label('Wavelength ($\mu$m)', fontsize = 18)                


# ax[1].tick_params(axis='both', which='major', labelsize=18)
# ax[1].set_xlabel('q (%)', fontsize = 18)
# ax[1].set_ylabel('u (%)', fontsize = 18)

# # plt.text(0.7,1.1,'SN', fontsize = 18)
# # plt.text(-2.5,0.3,'J2034 \n2017 June ', fontsize = 18)
# # plt.text(-2.7,-2.5,'BD+59 2267 \n2018 June ', fontsize = 18)


# #plt.errorbar(np.median(q_ISP)*100, np.median(u_ISP)*100)#, xerr = np.std(q_ISP)*100, yerr = np.std(u_ISP)*100)
# #plt.errorbar(np.median(q_ISP_2)*100, np.median(u_ISP_2)*100)#, xerr = np.std(q_ISP_2)*100, yerr =np.std(u_ISP_2)*100)

# # plt.plot(np.median(q_ISP)*100, np.median(u_ISP)*100, marker = 'x', markersize = 20, mew = 5, color = 'C0')
# # plt.plot(np.median(q_ISP_2)*100, np.median(u_ISP_2)*100, marker = 'x', markersize = 20, mew = 5, color = 'C1')

# # plt.text(-1.4, -0.8, "ISP\nmodel A", fontsize = 18, color = 'C0', ha = 'right')
# # plt.text(0.2, -1.9, "ISP model B", fontsize = 18, color = 'C1')

# #hightlight 0,0
# ax[1].plot([0,0], [-3,3], color = 'k', alpha = 0.3)
# ax[1].plot([-3,3],[0,0] , color = 'k', alpha = 0.3)


# ax[1].grid(ls = '--')
# ax[1].set_xlim([-3,3])
# ax[1].set_ylim([-3,3])

# ax[1].set_aspect('equal', adjustable='box')


# ########p theta
# ax[2].tick_params(axis='both', which='major', labelsize=18)
# ax[2].set_xlabel('Wavelength ($\mu$m)', fontsize = 18)
# ax[2].set_ylabel('p (%)', fontsize = 20, color = '#008018')


# # ax[2].plot(wlSol_smooth, p*100, marker = 'o', ls = 'None', alpha = 0.5)
# ax[2].errorbar(wlSol_smooth[good_ind], p_corr[good_ind]*100, dp[good_ind]*100, marker = 's', ls = 'None', color = '#008018')
# ax[2].set_xlim([wlMin, wlMax])
# ax[2].set_ylim(0,4)
# ax[2].tick_params(axis = 'y', labelcolor = '#008018')

# # good = ~np.isnan(p_corr)
# ax[2].plot(wlSol_smooth[~good_ind], 3*100*(dp[~good_ind]),  marker = 'v', ls = 'None', color = '#008018', ms = 10)


# t_ax = ax[2].twinx()
# t_ax.set_ylabel(r'$\theta$ ($\circ$)', fontsize = 20, color = '#86007D')
# t_ax.tick_params(axis='both', which='major', labelsize=18, labelcolor ='#86007D')

# t_ax.errorbar(wlSol_smooth[good_ind], thetadeg[good_ind], np.degrees(dtheta[good_ind]), marker = 'o', ls = 'None', color = '#86007D')
# t_ax.set_ylim([80,170])

# #flux
# ax[2].plot(wlSol, flux_spec/np.max(flux_spec)*2.5, color = '#FFA52C')
# ax[2].text(1.19,2.75,'WIRC+Pol Normalized Flux', color = '#FFA52C', fontsize = 18)



# plt.tight_layout(rect=[0, 0, 1, 0.95])

# plt.suptitle("SN2020oi (Ic)", fontsize = 24)

# plt.savefig('%s_summary_calibrated.pdf'%SN_name)

## Compute broadband q and u

In [ ]:
#Compute broadband values
q_bb = np.mean(cal_q_sm[plot_range])
q_bb_err = np.mean(cal_q_err_sm[plot_range])/np.sqrt(len(cal_q_err_sm[plot_range]))

u_bb = np.mean(cal_u_sm[plot_range])
u_bb_err = np.mean(cal_u_err_sm[plot_range])/np.sqrt(len(cal_u_err_sm[plot_range]))

In [ ]:
print(q_bb*100, q_bb_err*100, u_bb*100, u_bb_err*100)

In [ ]:
print(100*np.nanmean(p_corr[plot_range]), 100*np.mean(dp[SN_range])/np.sqrt(len(dp[plot_range])))

In [ ]:
p_bb = np.sqrt(q_bb**2 + u_bb**2)
dp_bb = 1/p_bb * np.sqrt( (q_bb * u_bb_err)**2 + (u_bb * q_bb_err)**2)

In [ ]:
print(p_bb*100, dp_bb*100, np.sqrt(p_bb**2 - dp_bb**2)*100)

## Typical uncertainty

In [ ]:
print(np.mean(cal_q_err_sm[plot_range])*100, np.mean(cal_u_err_sm[plot_range])*100)

In [ ]:
# plt.figure(figsize = (15,10))
# NIRES_range = np.logical_and(NIRES_spec['wl'] > 10000, NIRES_spec['wl'] < wlMax*14000)
# plt.plot(NIRES_spec['wl'][NIRES_range]/10000/(1+z_sn), NIRES_spec['flux'][NIRES_range]/np.max(NIRES_spec['flux'][NIRES_range])*3.7-2, color = 'g')

# plt.axvspan(1.18, 1.21, color = 'grey', alpha = 0.1)
# plt.axvspan(1.24, 1.27, color = 'grey', alpha = 0.1)

# #lines
# plt.axvline(1.282, color = 'b')

# plt.axvline(1.257, color = 'r')
